CPU times: total: 13d 23h 45min 35s
Wall time: 1d 4h 40min 3s

In [1]:
import pickle
import os
import datetime

In [2]:
import pandas as pd, numpy as np, matplotlib.pyplot as plt, scipy, time, warnings

import sys
from pathlib import Path

import sklearn
from sklearn.metrics import roc_auc_score, accuracy_score, f1_score

from evaluation_framework.pipeline import EvaluationPipeline
from evaluation_framework import steps as ef_steps
from evaluation_framework.datasets import get_datasets_for_pattern
from evaluation_framework.runners import run_on_many_datasets

In [3]:
from methods.EF_PAPE import calculate_PAPE_on_chunk
from methods.EF_TEST_SET_performance import calculate_TEST_SET_performance_on_chunk
from methods.EF_ATC import calculate_ATC_on_chunk
from methods.EF_DOC import calculate_DOC_on_chunk
from methods.EF_ImportanceWeighting import calculate_IW_on_chunk
from methods.EF_RT import calculate_RT_on_chunk
from methods.EF_CBPE import calculate_CBPE_on_chunk

from methods.utils import calculate_targets, save_results_to_df

In [4]:
import warnings 
warnings.filterwarnings('ignore')

In [5]:
# File folder with results, results will be stored here and loaded from there
results_files_folder = 'experiment_results'
if not os.path.exists(results_files_folder):
    os.makedirs(results_files_folder)

In [6]:
datasets = get_datasets_for_pattern(pattern="", data_dir="datasets")

In [7]:
for dataset in datasets:
    dataset.step_size = 2000    
    dataset.observations_in_chunk = 2000

In [8]:
len(datasets)

250

# EF Functions

In [9]:
methods = [
    "TEST_SET",
    "ATC",
    "DOC",
    "CBPE",
    "IW",
    'RT',
    "PAPE", 
          ]

In [10]:
monitored_models = [
    'LogisticRegression_',
    'LGBMClassifier_',
    'RandomForestClassifier_',
    'XGB_',
    'FT_Transformer_'
]

In [11]:
experiments = []

for method in methods:
    for monitored_model in monitored_models:
            experiments.append({
                'method':method,
                'monitored_model': monitored_model,
            })


In [14]:
len(experiments), len(datasets), len(datasets)*len(experiments)

(35, 250, 8750)

In [15]:
%%time
results = []
experiments_done = []
experiments_to_rerun = []

for i, experiment in enumerate(experiments[:]):
    method = experiment['method']
    monitored_model = experiment['monitored_model']
    calculation_steps_dic = {
    "TEST_SET": ef_steps.Step(description='Calculate TEST SET performance prediction on chunks', 
                     func=calculate_TEST_SET_performance_on_chunk, args=(monitored_model,)),
    "PAPE": ef_steps.Step(description='Calculate PAPE prediction on chunks', 
                     func=calculate_PAPE_on_chunk, args=(monitored_model,)),
    'ATC':  ef_steps.Step(description='Calculate ATC', 
                     func=calculate_ATC_on_chunk, args=(monitored_model, )),
    'DOC': ef_steps.Step(description='Calculate DOC', 
                     func=calculate_DOC_on_chunk, args=(monitored_model, 2000, 'random')),
    'IW': ef_steps.Step(description='Calculate IW', 
                      func=calculate_IW_on_chunk, args=(monitored_model,)),
    'RT': ef_steps.Step(description='Calculate RT', 
                        func=calculate_RT_on_chunk, args=(monitored_model,)),
    'CBPE': ef_steps.Step(description='Calculate CBPE prediction on chunks', 
                     func=calculate_CBPE_on_chunk, args=(monitored_model,)) 
    }
    
    calculation_step = calculation_steps_dic[method]
    print("experiment {} out of {}".format(i, len(experiments)))
    for j, dataset in enumerate(datasets):
        print("PROGRESS: " + str((i*len(datasets)+j)/len(experiments)/len(datasets)))
        print(method, monitored_model)

        
        pipeline = EvaluationPipeline(steps=[

            ef_steps.SplitDataStep(), #split raw to train test (reference/Production)
            ef_steps.CombineProcessedReferenceProductionWithRawStep(), # combine after processing
            ef_steps.SplitIntoChunksStep(), # split to chunks
            ef_steps.Step(description='Calculate targets', 
                     func=calculate_targets, args=(monitored_model, )), # custom step
            calculation_step,
            ef_steps.ComparePlotSingle(
                compare_data_entries_names=['zeros', 'roc', 'est_roc',],
                plot=False,
                scatter_entries= ['roc', 'est_roc'],
                show_legend=True, save_fig=False, fig_location='evaluation_plots_roc', fig_name=method + '_roc'),

            ef_steps.ComparePlotSingle(
                compare_data_entries_names=['zeros', 'accuracy', 'est_accuracy'],
                plot=False,
                scatter_entries= ['accuracy', 'est_accuracy'],
                show_legend=True, save_fig=False, fig_location='evaluation_plots_accuracy', fig_name=method+ '_accuracy'), 

            ef_steps.ComparePlotSingle(
                compare_data_entries_names=['zeros', 'f1', 'est_f1',],
                plot=False,
                scatter_entries=['f1', 'est_f1'],
                show_legend=True, save_fig=False, fig_location='evaluation_plots_f1', fig_name=method + '_f1'), 
            ])

        res, dat = run_on_many_datasets(datasets=[dataset], pipeline=pipeline, log_metrics=False,)
        
        
        if "traceback" in dat[0].keys():
            experiments_to_rerun.append((experiment, dataset.data_path, dat[0]['traceback']))
        else:
            df_results = save_results_to_df(dat, method, monitored_model)
            results.append(df_results)
            
        experiments_done.append(experiment)


experiment 0 out of 35
PROGRESS: 0.0
TEST_SET LogisticRegression_

Evaluating on: datasets\employment-2015_2016_2017_2018-AK.pq
Chunk size: 2000, all chunks: 10, reference_chunks: 3, transition chunks: 1, production_chunks (includes transition) :7
EVALUTING FOR:
LogisticRegression_
PROGRESS: 0.00011428571428571428
TEST_SET LogisticRegression_

Evaluating on: datasets\employment-2015_2016_2017_2018-AL.pq
Chunk size: 2000, all chunks: 71, reference_chunks: 23, transition chunks: 1, production_chunks (includes transition) :48
EVALUTING FOR:
LogisticRegression_
PROGRESS: 0.00022857142857142857
TEST_SET LogisticRegression_

Evaluating on: datasets\employment-2015_2016_2017_2018-AR.pq
Chunk size: 2000, all chunks: 45, reference_chunks: 14, transition chunks: 1, production_chunks (includes transition) :31
EVALUTING FOR:
LogisticRegression_
PROGRESS: 0.00034285714285714285
TEST_SET LogisticRegression_

Evaluating on: datasets\employment-2015_2016_2017_2018-AZ.pq
Chunk size: 2000, all chunks: 1

EVALUTING FOR:
LogisticRegression_
PROGRESS: 0.0034285714285714284
TEST_SET LogisticRegression_

Evaluating on: datasets\employment-2015_2016_2017_2018-NJ.pq
Chunk size: 2000, all chunks: 131, reference_chunks: 43, transition chunks: 1, production_chunks (includes transition) :88
EVALUTING FOR:
LogisticRegression_
PROGRESS: 0.0035428571428571427
TEST_SET LogisticRegression_

Evaluating on: datasets\employment-2015_2016_2017_2018-NM.pq
Chunk size: 2000, all chunks: 28, reference_chunks: 9, transition chunks: 1, production_chunks (includes transition) :19
EVALUTING FOR:
LogisticRegression_
PROGRESS: 0.003657142857142857
TEST_SET LogisticRegression_

Evaluating on: datasets\employment-2015_2016_2017_2018-NV.pq
Chunk size: 2000, all chunks: 42, reference_chunks: 13, transition chunks: 1, production_chunks (includes transition) :29
EVALUTING FOR:
LogisticRegression_
PROGRESS: 0.0037714285714285714
TEST_SET LogisticRegression_

Evaluating on: datasets\employment-2015_2016_2017_2018-NY.pq
Chu

EVALUTING FOR:
LogisticRegression_
PROGRESS: 0.006857142857142857
TEST_SET LogisticRegression_

Evaluating on: datasets\income-2015_2016_2017_2018-HI.pq
Chunk size: 2000, all chunks: 11, reference_chunks: 3, transition chunks: 1, production_chunks (includes transition) :8
EVALUTING FOR:
LogisticRegression_
PROGRESS: 0.006971428571428571
TEST_SET LogisticRegression_

Evaluating on: datasets\income-2015_2016_2017_2018-IA.pq
Chunk size: 2000, all chunks: 26, reference_chunks: 8, transition chunks: 1, production_chunks (includes transition) :18
EVALUTING FOR:
LogisticRegression_
PROGRESS: 0.0070857142857142855
TEST_SET LogisticRegression_

Evaluating on: datasets\income-2015_2016_2017_2018-ID.pq
Chunk size: 2000, all chunks: 12, reference_chunks: 3, transition chunks: 1, production_chunks (includes transition) :9
EVALUTING FOR:
LogisticRegression_
PROGRESS: 0.0072666
TEST_SET LogisticRegression_

Evaluating on: datasets\income-2015_2016_2017_2018-IL.pq
Chunk size: 2000, all chunks: 100, re

EVALUTING FOR:
LogisticRegression_
PROGRESS: 0.010514285714285714
TEST_SET LogisticRegression_

Evaluating on: datasets\income-2015_2016_2017_2018-TX.pq
Chunk size: 2000, all chunks: 200, reference_chunks: 65, transition chunks: 1, production_chunks (includes transition) :135
EVALUTING FOR:
LogisticRegression_
PROGRESS: 0.010628571428571428
TEST_SET LogisticRegression_

Evaluating on: datasets\income-2015_2016_2017_2018-UT.pq
Chunk size: 2000, all chunks: 23, reference_chunks: 7, transition chunks: 1, production_chunks (includes transition) :16
EVALUTING FOR:
LogisticRegression_
PROGRESS: 0.010742857142857143
TEST_SET LogisticRegression_

Evaluating on: datasets\income-2015_2016_2017_2018-VA.pq
Chunk size: 2000, all chunks: 68, reference_chunks: 22, transition chunks: 1, production_chunks (includes transition) :46
EVALUTING FOR:
LogisticRegression_
PROGRESS: 0.010857142857142857
TEST_SET LogisticRegression_

Evaluating on: datasets\income-2015_2016_2017_2018-VT.pq
Chunk size: 2000, all

EVALUTING FOR:
LogisticRegression_
PROGRESS: 0.013942857142857142
TEST_SET LogisticRegression_

Evaluating on: datasets\medicalcov-2015_2016_2017_2018-MN.pq
Chunk size: 2000, all chunks: 25, reference_chunks: 8, transition chunks: 1, production_chunks (includes transition) :17
EVALUTING FOR:
LogisticRegression_
PROGRESS: 0.014057142857142857
TEST_SET LogisticRegression_

Evaluating on: datasets\medicalcov-2015_2016_2017_2018-MO.pq
Chunk size: 2000, all chunks: 34, reference_chunks: 11, transition chunks: 1, production_chunks (includes transition) :23
EVALUTING FOR:
LogisticRegression_
PROGRESS: 0.014171428571428571
TEST_SET LogisticRegression_

Evaluating on: datasets\medicalcov-2015_2016_2017_2018-MS.pq
Chunk size: 2000, all chunks: 18, reference_chunks: 6, transition chunks: 1, production_chunks (includes transition) :12
EVALUTING FOR:
LogisticRegression_
PROGRESS: 0.014285714285714287
TEST_SET LogisticRegression_

Evaluating on: datasets\medicalcov-2015_2016_2017_2018-MT.pq
Chunk si

...

PROGRESS: 0.9788571428571429
PAPE FT_Transformer_

Evaluating on: datasets\income-2015_2016_2017_2018-KS.pq
Chunk size: 2000, all chunks: 23, reference_chunks: 7, transition chunks: 1, production_chunks (includes transition) :16
PROGRESS: 0.9789714285714285
PAPE FT_Transformer_

Evaluating on: datasets\income-2015_2016_2017_2018-KY.pq
Chunk size: 2000, all chunks: 32, reference_chunks: 10, transition chunks: 1, production_chunks (includes transition) :22
PROGRESS: 0.9790857142857143
PAPE FT_Transformer_

Evaluating on: datasets\income-2015_2016_2017_2018-LA.pq
Chunk size: 2000, all chunks: 31, reference_chunks: 10, transition chunks: 1, production_chunks (includes transition) :21
PROGRESS: 0.9792000000000001
PAPE FT_Transformer_

Evaluating on: datasets\income-2015_2016_2017_2018-MA.pq
Chunk size: 2000, all chunks: 59, reference_chunks: 19, transition chunks: 1, production_chunks (includes transition) :40
PROGRESS: 0.9793142857142857
PAPE FT_Transformer_

Evaluating on: datasets\income

PROGRESS: 0.9829714285714285
PAPE FT_Transformer_

Evaluating on: datasets\medicalcov-2015_2016_2017_2018-AL.pq
Chunk size: 2000, all chunks: 28, reference_chunks: 9, transition chunks: 1, production_chunks (includes transition) :19
PROGRESS: 0.9830857142857143
PAPE FT_Transformer_

Evaluating on: datasets\medicalcov-2015_2016_2017_2018-AR.pq
Chunk size: 2000, all chunks: 18, reference_chunks: 6, transition chunks: 1, production_chunks (includes transition) :12
PROGRESS: 0.9832000000000001
PAPE FT_Transformer_

Evaluating on: datasets\medicalcov-2015_2016_2017_2018-AZ.pq
Chunk size: 2000, all chunks: 38, reference_chunks: 13, transition chunks: 1, production_chunks (includes transition) :25
PROGRESS: 0.9833142857142857
PAPE FT_Transformer_

Evaluating on: datasets\medicalcov-2015_2016_2017_2018-CA.pq
Chunk size: 2000, all chunks: 213, reference_chunks: 72, transition chunks: 1, production_chunks (includes transition) :141
PROGRESS: 0.9834285714285714
PAPE FT_Transformer_

Evaluating on

PROGRESS: 0.9870857142857143
PAPE FT_Transformer_

Evaluating on: datasets\medicalcov-2015_2016_2017_2018-PA.pq
Chunk size: 2000, all chunks: 67, reference_chunks: 22, transition chunks: 1, production_chunks (includes transition) :45
PROGRESS: 0.9872000000000001
PAPE FT_Transformer_

Evaluating on: datasets\medicalcov-2015_2016_2017_2018-RI.pq
Chunk size: 2000, all chunks: 5, reference_chunks: 1, transition chunks: 1, production_chunks (includes transition) :4
PROGRESS: 0.9873142857142857
PAPE FT_Transformer_

Evaluating on: datasets\medicalcov-2015_2016_2017_2018-SC.pq
Chunk size: 2000, all chunks: 28, reference_chunks: 9, transition chunks: 1, production_chunks (includes transition) :19
PROGRESS: 0.9874285714285714
PAPE FT_Transformer_

Evaluating on: datasets\medicalcov-2015_2016_2017_2018-SD.pq
Chunk size: 2000, all chunks: 4, reference_chunks: 1, transition chunks: 1, production_chunks (includes transition) :3
PROGRESS: 0.9875428571428571
PAPE FT_Transformer_

Evaluating on: datas

PROGRESS: 0.9912000000000001
PAPE FT_Transformer_

Evaluating on: datasets\mobility-2015_2016_2017_2018-MO.pq
Chunk size: 2000, all chunks: 17, reference_chunks: 5, transition chunks: 1, production_chunks (includes transition) :12
PROGRESS: 0.9913142857142857
PAPE FT_Transformer_

Evaluating on: datasets\mobility-2015_2016_2017_2018-MS.pq
Chunk size: 2000, all chunks: 7, reference_chunks: 2, transition chunks: 1, production_chunks (includes transition) :5
PROGRESS: 0.9914285714285714
PAPE FT_Transformer_

Evaluating on: datasets\mobility-2015_2016_2017_2018-MT.pq
could not execute step Split preprocessed data into chunks that will be points for monit model: index -1 is out of bounds for axis 0 with size 0
PROGRESS: 0.9915428571428571
PAPE FT_Transformer_

Evaluating on: datasets\mobility-2015_2016_2017_2018-NC.pq
Chunk size: 2000, all chunks: 28, reference_chunks: 9, transition chunks: 1, production_chunks (includes transition) :19
PROGRESS: 0.9916571428571428
PAPE FT_Transformer_

Eva

PROGRESS: 0.9993142857142857
PAPE FT_Transformer_

Evaluating on: datasets\traveltime-2015_2016_2017_2018-VA.pq
Chunk size: 2000, all chunks: 59, reference_chunks: 19, transition chunks: 1, production_chunks (includes transition) :40
PROGRESS: 0.9994285714285714
PAPE FT_Transformer_

Evaluating on: datasets\traveltime-2015_2016_2017_2018-VT.pq
Chunk size: 2000, all chunks: 4, reference_chunks: 1, transition chunks: 1, production_chunks (includes transition) :3
PROGRESS: 0.9995428571428571
PAPE FT_Transformer_

Evaluating on: datasets\traveltime-2015_2016_2017_2018-WA.pq
Chunk size: 2000, all chunks: 50, reference_chunks: 16, transition chunks: 1, production_chunks (includes transition) :34
PROGRESS: 0.9996571428571428
PAPE FT_Transformer_

Evaluating on: datasets\traveltime-2015_2016_2017_2018-WI.pq
Chunk size: 2000, all chunks: 43, reference_chunks: 14, transition chunks: 1, production_chunks (includes transition) :29
PROGRESS: 0.9997714285714286
PAPE FT_Transformer_

Evaluating on: d

In [16]:
df = pd.concat(results).reset_index(drop=True)

In [17]:
date = datetime.datetime.now().strftime("%Y_%m_%d_%H_%M_%S")
date

'2024_05_18_14_53_59'

In [18]:
df

,method,dataset,monitored_model,accuracy,est_accuracy,roc_auc,est_roc_auc,f1,est_f1,period,chunksize,step
0,TEST_SET,employment-2015_2016_2017_2018-AK.pq,LogisticRegression_,0.7870,0.791343,0.872960,0.875932,0.745823,0.756089,reference,2000,2000
1,TEST_SET,employment-2015_2016_2017_2018-AK.pq,LogisticRegression_,0.7940,0.791343,0.879778,0.875932,0.757075,0.756089,reference,2000,2000
2,TEST_SET,employment-2015_2016_2017_2018-AK.pq,LogisticRegression_,0.7900,0.791343,0.871443,0.875932,0.764574,0.756089,reference,2000,2000
3,TEST_SET,employment-2015_2016_2017_2018-AK.pq,LogisticRegression_,0.7810,0.791343,0.868667,0.875932,0.744457,0.756089,transition,2000,2000
4,TEST_SET,employment-2015_2016_2017_2018-AK.pq,LogisticRegression_,0.7840,0.791343,0.865775,0.875932,0.753986,0.756089,production,2000,2000
...,...,...,...,...,...,...,...,...,...,...,...,...
413047,PAPE,traveltime-2015_2016_2017_2018-WV.pq,FT_Transformer_,0.5880,0.611882,0.554531,0.571124,0.000000,0.000000,production,2000,2000
413048,PAPE,traveltime-2015_2016_2017_2018-WV.pq,FT_Transformer_,0.6125,0.619228,0.581644,0.569099,0.000000,0.000000,production,2000,2000
413049,PAPE,traveltime-2015_2016_2017_2018-WY.pq,FT_Transformer_,0.8010,0.801363,0.538060,0.531612,0.000000,0.000000,reference,2000,2000
413050,PAPE,traveltime-2015_2016_2017_2018-WY.pq,FT_Transformer_,0.7990,0.806519,0.508025,0.516008,0.000000,0.000000,transition,2000,2000


In [19]:
results_files_folder

'experiment_results'

In [20]:
date

'2024_05_18_14_53_59'

In [21]:
df.to_parquet(os.path.join(results_files_folder, 'experiment_results.pq'))
